In [ ]:
# coding=utf-8
# Copyright 2018 The Google AI Language Team Authors and The HugginFace Inc. team.
# Copyright (c) 2018, NVIDIA CORPORATION.  All rights reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#	 http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
"""BERT finetuning runner."""

from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import csv
import gc
import re
import pandas as pd
# import emoji
import os
import logging
import argparse
import random
import regex as re
from tqdm import tqdm, trange
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import torch
import copy
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from torch.utils.data.distributed import DistributedSampler

from huggingface_hub import login
from transformers import AutoTokenizer, AutoModelForCausalLM, GemmaConfig

from sklearn.metrics import accuracy_score,confusion_matrix,recall_score
from sklearn.metrics import classification_report

In [ ]:
import os
import numpy as np

logging.basicConfig(format = '%(asctime)s - %(levelname)s - %(name)s -   %(message)s',
					datefmt = '%m/%d/%Y %H:%M:%S',
					level = logging.INFO)
logger = logging.getLogger(__name__)

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
# torch.set_grad_enabled(False)
torch.set_grad_enabled(True)
device

device(type='cuda')

In [ ]:
def unlearning(net, forget, forget_neu, device):
    # net.half() helped the model to run on the cluster,
    # even though it ended up stopping execution after 24 hours.
    tox_shapley_values = calculate_shapley_values_fa(net, forget, device, 10) # og is 10
    # Save values to help save memory.
    torch.save(tox_shapley_values, "tox_shapley_values.pt")
    del tox_shapley_values
    nontox_shapley_values = calculate_shapley_values_fa(net, forget_neu, device, 10) # og is 10

    tox_shapley_values = torch.load("tox_shapley_values.pt")
    diff_shap_values_toxnontox = tox_shapley_values - nontox_shapley_values


    log_file = open("log_shapley_values.txt", "w")  # Open a log file for writing
    print("shapley values", diff_shap_values_toxnontox)
    log_file.write("Shapley Values: {}\n".format(diff_shap_values_toxnontox))

    max_diff_shap_values_ind = np.argpartition(diff_shap_values_toxnontox, -150)[-150:]

    diff_shap_values = diff_shap_values_toxnontox[max_diff_shap_values_ind]  # Define 'diff_shap_values'
    log_file.write("Top 10 Shapley Values: {}\n".format(diff_shap_values))

    model_arr, model_slist = get_net_arr(net)
    model_arr[max_diff_shap_values_ind] = 0
    updated_model = get_arr_net(net, model_arr, model_slist)  # Assign 'updated_model'

    return updated_model  # Add a return statement to return the updated model

def calculate_shapley_values_fa(model, data_loader, device, repeats=100): 
  print("Calculating Shapley Values...")
  model_arr, model_slist = get_net_arr(model)
  num_neurons = len(model_arr)
  print(num_neurons) # added this
  # num_neurons = 2614341888
  shapley_values = torch.zeros(num_neurons)  # Initialize Shapley values for each neuron # Added device=device but cuda oom issue
  # removed the .numpy() here.
  torch.save(shapley_values, "shapley_values.pt")
  del shapley_values
  gc.collect()
  torch.cuda.empty_cache()
  print("shapley values")

  for input_ids, input_mask, segment_ids, label_ids in tqdm(data_loader, desc="Calculating Shapley Values"):
    input_ids = input_ids.to(device)
    # print("input ids")
    input_mask = input_mask.to(device)
    # print("input_mask")
    segment_ids = segment_ids.to(device)
    label_ids = label_ids.to(device)

    # Print the contents of input_ids, input_mask, segment_ids, and label_ids

    print("input_ids:", len(input_ids))
    print("input_mask:", len(input_mask))
    print("segment_ids:", len(segment_ids))
    print("label_ids:", len(label_ids))

# for x, y in data_loader:
    # device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu") # this was originally commented out

    # x, y = x.to(device), y.to(device) # this was originally commented out
    for i in range(repeats):
      print(f"Iteration {i+1}/{repeats}")
      perm = random.sample(range(num_neurons), int(num_neurons*0.01))  # Randomly sample a permutation but original is 0.25

      # Set all neurons to zero except the ones in the current permutation
      zeroed_neurons = torch.ones(num_neurons) # adding device=device results in an error
      zeroed_neurons[list(perm)] = 0
      zeroed_model = np.multiply(model_arr, zeroed_neurons.numpy())
      del zeroed_neurons # added this to see if memory improves
      del perm # added this to see if memory improves
      # print("zeroed_model", zeroed_model)
      gc.collect()
      torch.cuda.empty_cache()
      #zeroed_model = sim.get_arr_net(model, zeroed_model, model_slist)
      zeroed_model = get_arr_net(model, zeroed_model, model_slist)
      zeroed_model.eval() # GPT suggestion
      # Compute the output with the zeroed neurons
      zeroed_output = zeroed_model(input_ids=input_ids, attention_mask=input_mask) #, labels=label_ids) # changed this based on gpt
      # print(zeroed_output.shape)
      # print("zeroed output", type(zeroed_output))


      # Taking the softmax output

      # zeroed_output_soft = F.log_softmax(zeroed_output, dim=1) # GPT said to take out
      # zeroed_model.zero_grad() #  GPT suggestions
      zeroed_output_logits = zeroed_output.logits
      del zeroed_output
      # print(zeroed_output_logits.shape)
      # print(label_ids.shape)

      logits = zeroed_output_logits[:, -1, :]  # GPT suggestion for final token classification
      # print("logits", type(logits), logits.requires_grad)

      # loss = F.cross_entropy(logits, label_ids) # GPT: softmax is included.
      loss = F.cross_entropy(logits, label_ids)
      zeroed_model.zero_grad() # GPT suggestion
      loss.backward()
      print("after backward function: passed!!")

      prev_index = 0
      index = 0

      # Load back the Shapley values
      shapley_values = torch.load("shapley_values.pt")

      for param in zeroed_model.parameters():
        prev_index = index
        index = index + len(param.flatten())
        if param.grad != None:
            #shapley_values[prev_index:index] = shapley_values[prev_index:index] + np.abs(param.grad.detach().numpy().flatten() * model_arr[prev_index:index])
            grad_np = param.grad.cpu().detach().numpy().flatten()
            shapley_values[prev_index:index] = shapley_values[prev_index:index] + np.abs(
                grad_np * model_arr[prev_index:index]
            )
      print("end of loop: passed!")
      print("shap val type: ", type(shapley_values))
      gc.collect()
      torch.cuda.empty_cache()
  return shapley_values

def get_arr_net(_model, arr, slist):
  arr = torch.from_numpy(arr).unsqueeze(1)
  arr = arr.numpy()
  # print(arr[-5:])

  _param_list = []
  start_index = 0
  for shape in slist:
      #end_index = start_index + nd.prod(list(shape))
      end_index = start_index + np.prod(list(shape))
      item = arr[start_index:end_index]
      start_index = end_index
      item = item.reshape(shape)
      _param_list.append(item)

  params = _model.state_dict().copy()
  print("_param_list", len(_param_list))
  with torch.no_grad():
      _index = 0
      for name in params:
        if _index != 288: # adding this because the params has one extra item - lm_head.weight, which is not included in .parameters.
          if "weight" in name or "bias" in name:
            # print(name, _index)
            params[name] = torch.from_numpy(_param_list[_index])
            _index = _index + 1

  model = copy.deepcopy(_model)
  del _model
  gc.collect()
  torch.cuda.empty_cache()
  model.load_state_dict(params, strict=False)

  return model

def get_net_arr(model):
  # param_list = [param.data.numpy() for param in model.parameters()]
  param_list = [param.data.cpu().numpy() for param in model.parameters()]  # Move to CPU and then convert to NumPy
  print("param_list", len(param_list)) #added this

  #arr = nd.array([[]])
  arr = np.array([[]]) 
  slist = []
  counter = 0
  for index, item in enumerate(param_list):
    counter += 1
    print("index", counter)
    slist.append(item.shape)
    item = item.reshape((-1, 1))
    if index == 0:
        arr = item
        # print("arr is item")
    else:
        #arr = nd.concatenate((arr, item), axis=0)

        arr = np.concatenate((arr, item), axis=0) 

        # print("arr is concatenated", (arr, item))

  #arr = nd.array(arr).squeeze()
  arr = np.array(arr).squeeze()
  print("arr") # added this
  print("slist")
  print(slist)

  return arr, slist

In [ ]:
class InputExample(object):
	"""A single training/test example for simple sequence classification."""

	def __init__(self, guid, text_a, text_b=None, label=None):
		"""Constructs a InputExample.
		Args:
			guid: Unique id for the example.
			text_a: string. The untokenized text of the first sequence. For single
			sequence tasks, only this sequence must be specified.
			text_b: (Optional) string. The untokenized text of the second sequence.
			Only must be specified for sequence pair tasks.
			label: (Optional) string. The label of the example. This should be
			specified for train and dev examples, but not for test examples.
		"""
		self.guid = guid
		self.text_a = text_a
		self.text_b = text_b
		self.label = label


class InputFeatures(object):
	"""A single set of features of data."""

	def __init__(self, input_ids, input_mask, segment_ids, label_id):
		self.input_ids = input_ids
		self.input_mask = input_mask
		self.segment_ids = segment_ids
		self.label_id = label_id

In [ ]:
class DataProcessor(object):
	"""Base class for data converters for sequence classification data sets."""

	def get_train_examples(self, data_dir):
		"""Gets a collection of `InputExample`s for the train set."""
		raise NotImplementedError()

	def get_dev_examples(self, data_dir):
		"""Gets a collection of `InputExample`s for the dev set."""
		raise NotImplementedError()
	def get_test_examples(self, data_dir):
		"""Gets a collection of `InputExample`s for the dev set."""
		raise NotImplementedError()

	def get_forget_examples(self, data_dir):
		"""Gets a collection of `InputExample`s for the dev set."""
		raise NotImplementedError()
	def get_forget_neu_examples(self, data_dir):
		"""Gets a collection of `InputExample`s for the dev set."""
		raise NotImplementedError()


	def get_labels(self):
		"""Gets the list of labels for this data set."""
		raise NotImplementedError()

	@classmethod
	def _read_tsv(cls, input_file, quotechar=None):
		"""Reads a tab separated value file."""
		with open(input_file, "r", encoding='utf-8') as f:
			reader = csv.reader(f, delimiter="\t", quotechar=quotechar)
			lines = []
			for line in reader:
				lines.append(line)
			return lines

In [ ]:
class SentiProcessor(DataProcessor):
	"""Processor for Senti dataset."""

	def get_train_examples(self, data_dir):
		"""See base class."""
		return self._create_examples(
			pd.read_csv(os.path.join(data_dir, "train.tsv"),sep='\t'), "train")

	def get_dev_examples(self, data_dir):
		"""See base class."""
		return self._create_examples(
			pd.read_csv(os.path.join(data_dir, "dev.tsv"),sep='\t'), "dev")

	def get_test_examples(self, data_dir):
		"""See base class."""
		return self._create_examples(
			pd.read_csv(os.path.join(args.template_dir, "sentence_template.csv"),sep=','), "test")

	# def get_forget_examples(self, data_dir):
	# 	"""See base class."""
	# 	return self._create_examples(
	# 		pd.read_csv(os.path.join("logs_unlearning/experiment1/", "false_pos_lesbian.csv"), sep=','), "forget")

	def get_forget_examples(self, data_dir):
		"""See base class."""
		return self._create_examples(pd.read_csv("african_df.csv"), "forget")

	def get_forget_neu_examples(self, data_dir):
		"""See base class."""
		return self._create_examples(pd.read_csv("cf_african_df.csv"), "forgetneu")

	def get_labels(self):
		"""See base class."""
		# return ["TRUE","FALSE"]
		return ["NONTOXIC","TOXIC"]

	def twitter_tokenizer(self, line):
		"""Preprocess the tweet texts"""
		line = str(line)
		line = line.lower()
		# line = emoji.demojize(line) I took this out because of import issues
		line = re.sub(r'http\S+', ' ', line)
		line = re.sub('@[\w_]+', ' ', line)
		line = re.sub('\|LBR\|', '', line)
		line = re.sub('\.\.\.+', ' ', line)
		line = re.sub('!!+', '!', line)
		line = re.sub('\?\?+', '?', line)
		return line

	def _create_examples(self, data, set_type):
		"""Creates examples for the training and dev sets."""
		examples = []
		if(set_type=="forget"):

				# l=data[data.keyword=="african"].head(70)

				# print("len", len(l), l.columns)
				print(data.head())

				for k in range(len(data)):
					guid = "%s-%s" % (set_type, k)
					# text_a=self.twitter_tokenizer(l['comment'].iloc[k])
					text_a = data["phrase"].iloc[k]
					# label=l['is_toxic'].iloc[k]
					label = data["toxicity"].iloc[k]

					examples.append(
						InputExample(guid=guid, text_a=text_a, text_b=None, label=label))
		elif(set_type=="forgetneu"):
		# elif(set_type=="forgetneu"):

				# l=data[data.keyword=="african"].head(70)

				# print("len", len(l), l.columns)

				# for k in range(len(l)):
				# 	guid = "%s-%s" % (set_type, k)
				# 	text_a=self.twitter_tokenizer(l['comment'].iloc[k]).strip()
				# 	label=l['is_toxic'].iloc[k]

				# 	text_a=text_a.replace("african","")
				# 	print("textttt", text_a)
				print(data.head())

				for k in range(len(data)):
					guid = "%s-%s" % (set_type, k)
					# text_a=self.twitter_tokenizer(l['comment'].iloc[k])
					text_a = data["phrase"].iloc[k]
					# label=l['is_toxic'].iloc[k]
					label = data["toxicity"].iloc[k]

					examples.append(
						InputExample(guid=guid, text_a=text_a, text_b=None, label=label))
		else:
			# for i in range(len(data)):
			# 	guid = "%s-%s" % (set_type, i)
			# 	text_a = self.twitter_tokenizer(data['comment'].loc[i])
			# 	label = data['is_toxic'].loc[i]
			# 	# print("set_type", set_type)
				print(data.head())

				for k in range(len(data)):
					guid = "%s-%s" % (set_type, k)
					# text_a=self.twitter_tokenizer(l['comment'].iloc[k])
					text_a = data["phrase"].iloc[k]
					# label=l['is_toxic'].iloc[k]
					label = data["toxicity"].iloc[k]

				examples.append(
					InputExample(guid=guid, text_a=text_a, text_b=None, label=label))
		return examples

<>:43: SyntaxWarning: invalid escape sequence '\w'
<>:44: SyntaxWarning: invalid escape sequence '\|'
<>:45: SyntaxWarning: invalid escape sequence '\.'
<>:47: SyntaxWarning: invalid escape sequence '\?'
<>:43: SyntaxWarning: invalid escape sequence '\w'
<>:44: SyntaxWarning: invalid escape sequence '\|'
<>:45: SyntaxWarning: invalid escape sequence '\.'
<>:47: SyntaxWarning: invalid escape sequence '\?'
/tmp/ipython-input-2525961080.py:43: SyntaxWarning: invalid escape sequence '\w'
  line = re.sub('@[\w_]+', ' ', line)
/tmp/ipython-input-2525961080.py:44: SyntaxWarning: invalid escape sequence '\|'
  line = re.sub('\|LBR\|', '', line)
/tmp/ipython-input-2525961080.py:45: SyntaxWarning: invalid escape sequence '\.'
  line = re.sub('\.\.\.+', ' ', line)
/tmp/ipython-input-2525961080.py:47: SyntaxWarning: invalid escape sequence '\?'
  line = re.sub('\?\?+', '?', line)


In [ ]:
def convert_examples_to_features(examples, label_list, max_seq_length, tokenizer):
	"""Loads a data file into a list of `InputBatch`s."""

	label_map = {label : i for i, label in enumerate(label_list)}

	features = []
	for (ex_index, example) in enumerate(examples):
		tokens_a = tokenizer.tokenize(example.text_a)
		#logger.info("example:", example)
		tokens_b = None
		if example.text_b:
			tokens_b = tokenizer.tokenize(example.text_b)
			# Modifies `tokens_a` and `tokens_b` in place so that the total
			# length is less than the specified length.
			# Account for [CLS], [SEP], [SEP] with "- 3"
			_truncate_seq_pair(tokens_a, tokens_b, max_seq_length - 3)
		else:
			# Account for [CLS] and [SEP] with "- 2"
			if len(tokens_a) > max_seq_length - 2:
				tokens_a = tokens_a[:(max_seq_length - 2)]

		# The convention in BERT is:
		# (a) For sequence pairs:
		#  tokens:   [CLS] is this jack ##son ##ville ? [SEP] no it is not . [SEP]
		#  type_ids: 0   0  0	0	0	 0	   0 0	1  1  1  1   1 1
		# (b) For single sequences:
		#  tokens:   [CLS] the dog is hairy . [SEP]
		#  type_ids: 0   0   0   0  0	 0 0
		#
		# Where "type_ids" are used to indicate whether this is the first
		# sequence or the second sequence. The embedding vectors for `type=0` and
		# `type=1` were learned during pre-training and are added to the wordpiece
		# embedding vector (and position vector). This is not *strictly* necessary
		# since the [SEP] token unambigiously separates the sequences, but it makes
		# it easier for the model to learn the concept of sequences.
		#
		# For classification tasks, the first vector (corresponding to [CLS]) is
		# used as as the "sentence vector". Note that this only makes sense because
		# the entire model is fine-tuned.
		tokens = ["[CLS]"] + tokens_a + ["[SEP]"]
		segment_ids = [0] * len(tokens)

		if tokens_b:
			tokens += tokens_b + ["[SEP]"]
			segment_ids += [1] * (len(tokens_b) + 1)

		input_ids = tokenizer.convert_tokens_to_ids(tokens)

		# The mask has 1 for real tokens and 0 for padding tokens. Only real
		# tokens are attended to.
		input_mask = [1] * len(input_ids)

		# Zero-pad up to the sequence length.
		padding = [0] * (max_seq_length - len(input_ids))
		input_ids += padding
		input_mask += padding
		segment_ids += padding

		assert len(input_ids) == max_seq_length
		assert len(input_mask) == max_seq_length
		assert len(segment_ids) == max_seq_length
		#print("example.label:", example.label)
		label_id = label_map[str(example.label).upper()]

		if ex_index < 5:
			logger.info("*** Example ***")
			logger.info("guid: %s" % (example.guid))
			logger.info("tokens: %s" % " ".join(
					[str(x) for x in tokens]))
			logger.info("input_ids: %s" % " ".join([str(x) for x in input_ids]))
			logger.info("input_mask: %s" % " ".join([str(x) for x in input_mask]))
			logger.info(
					"segment_ids: %s" % " ".join([str(x) for x in segment_ids]))
			logger.info("label: %s (id = %d)" % (example.label, label_id))

		features.append(
				InputFeatures(input_ids=input_ids,
							  input_mask=input_mask,
							  segment_ids=segment_ids,
							  label_id=label_id))
	return features

In [ ]:
def _truncate_seq_pair(tokens_a, tokens_b, max_length):
	"""Truncates a sequence pair in place to the maximum length."""

	# This is a simple heuristic which will always truncate the longer sequence
	# one token at a time. This makes more sense than truncating an equal percent
	# of tokens from each, since if one sequence is very short then each token
	# that's truncated likely contains more information than a longer sequence.
	while True:
		total_length = len(tokens_a) + len(tokens_b)
		if total_length <= max_length:
			break
		if len(tokens_a) > len(tokens_b):
			tokens_a.pop()
		else:
			tokens_b.pop()

def accuracy(out, labels):
	outputs = np.argmax(out, axis=1)
	return np.sum(outputs == labels)

def warmup_linear(x, warmup=0.002):
	if x < warmup:
		return x/warmup
	return 1.0 - x

In [ ]:
login(token="")

In [ ]:
# Load model
tokenizer = AutoTokenizer.from_pretrained("google/gemma-2-2b-it")
model = AutoModelForCausalLM.from_pretrained("google/gemma-2-2b-it").to(device)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/47.0k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

...

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/241M [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.99G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/187 [00:00<?, ?B/s]

In [ ]:
print(model.state_dict)

<bound method Module.state_dict of Gemma2ForCausalLM(
  (model): Gemma2Model(
    (embed_tokens): Embedding(256000, 2304, padding_idx=0)
    (layers): ModuleList(
      (0-25): 26 x Gemma2DecoderLayer(
        (self_attn): Gemma2Attention(
          (q_proj): Linear(in_features=2304, out_features=2048, bias=False)
          (k_proj): Linear(in_features=2304, out_features=1024, bias=False)
          (v_proj): Linear(in_features=2304, out_features=1024, bias=False)
          (o_proj): Linear(in_features=2048, out_features=2304, bias=False)
        )
        (mlp): Gemma2MLP(
          (gate_proj): Linear(in_features=2304, out_features=9216, bias=False)
          (up_proj): Linear(in_features=2304, out_features=9216, bias=False)
          (down_proj): Linear(in_features=9216, out_features=2304, bias=False)
          (act_fn): PytorchGELUTanh()
        )
        (input_layernorm): Gemma2RMSNorm((2304,), eps=1e-06)
        (post_attention_layernorm): Gemma2RMSNorm((2304,), eps=1e-06)
      

In [ ]:
processors = {

  "senti": SentiProcessor,
}

num_labels_task = {

  "senti": 2,
}

In [ ]:
task_name = "african_unlearning"

In [ ]:
processor = processors["senti"]()
num_labels = num_labels_task["senti"]
label_list = processor.get_labels()

In [ ]:
label_list

['NONTOXIC', 'TOXIC']

In [ ]:
processors

{'senti': __main__.SentiProcessor}

In [ ]:
forget_examples = processor.get_forget_examples("/content/")
print("forget_examples", len(forget_examples))
forget_features = convert_examples_to_features(
forget_examples, label_list, 128, tokenizer)
logger.info("***** Running evaluation *****")
logger.info("  Num examples = %d", len(forget_examples), len(forget_features))
logger.info("  Batch size = %d", 8)
all_fg_input_ids = torch.tensor([f.input_ids for f in forget_features], dtype=torch.long, device=device)
all_fg_input_mask = torch.tensor([f.input_mask for f in forget_features], dtype=torch.long, device=device)
all_fg_segment_ids = torch.tensor([f.segment_ids for f in forget_features], dtype=torch.long, device=device)
all_fg_label_ids = torch.tensor([f.label_id for f in forget_features], dtype=torch.long, device=device)
forget_data = TensorDataset(all_fg_input_ids, all_fg_input_mask, all_fg_segment_ids, all_fg_label_ids)

forget_sampler = SequentialSampler(forget_data)
forget_dataloader = DataLoader(forget_data, sampler=forget_sampler, batch_size=8)

   Unnamed: 0.1  Unnamed: 0   template  toxicity                      phrase  \
0           181         181   name_adj  nontoxic      Thel is a nice african   
1           223         223   name_adj     toxic   Ziv is a terrible african   
2           239         239  being_adj     toxic     Being african is stupid   
3           323         323   name_adj     toxic      Sara is a ugly african   
4           359         359   name_adj     toxic  Morgan is a fucked african   

   model pred  
0           1  
1           1  
2           1  
3           1  
4           3  
forget_examples 1514


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
forget_neu_examples = processor.get_forget_neu_examples("/content/") # originally, arg.data_dir
print("forget_examples", len(forget_neu_examples))
forget_neu_features = convert_examples_to_features(
forget_neu_examples, label_list, 128, tokenizer)
logger.info("***** Running evaluation *****")
logger.info("  Num examples = %d", len(forget_neu_examples), len(forget_neu_features))
logger.info("  Batch size = %d", 8)
all_fg_neu_input_ids = torch.tensor([f.input_ids for f in forget_neu_features], dtype=torch.long, device=device)
all_fg_neu_input_mask = torch.tensor([f.input_mask for f in forget_neu_features], dtype=torch.long, device=device)
all_fg_neu_segment_ids = torch.tensor([f.segment_ids for f in forget_neu_features], dtype=torch.long, device=device)
all_fg_neu_label_ids = torch.tensor([f.label_id for f in forget_neu_features], dtype=torch.long, device=device)
forget_neu_data = TensorDataset(all_fg_neu_input_ids, all_fg_neu_input_mask, all_fg_neu_segment_ids, all_fg_neu_label_ids)

forget_neu_sampler = SequentialSampler(forget_neu_data)
forget_neu_dataloader = DataLoader(forget_neu_data, sampler=forget_neu_sampler, batch_size=8)

   Unnamed: 0.1  Unnamed: 0   template  toxicity              phrase  \
0           181         181   name_adj  nontoxic      Thel is a nice   
1           223         223   name_adj     toxic   Ziv is a terrible   
2           239         239  being_adj     toxic     Being is stupid   
3           323         323   name_adj     toxic      Sara is a ugly   
4           359         359   name_adj     toxic  Morgan is a fucked   

   model pred  
0           1  
1           1  
2           1  
3           1  
4           3  
forget_examples 1514


In [ ]:
# Calculate the values

model = unlearning(model, forget_dataloader, forget_neu_dataloader, device)

model_to_save = model.module if hasattr(model, 'module') else model  # Only save the model it-self
torch.save(model_to_save, 'afr_model_dict.pt')

Calculating Shapley Values...
param_list 288
index 1
index 2
index 3
index 4
index 5
index 6
index 7
index 8
index 9
index 10
index 11
index 12
index 13
index 14
index 15
index 16
index 17
index 18
index 19
index 20
index 21
index 22
index 23
index 24
index 25
index 26
index 27
index 28
index 29
index 30
index 31
index 32
index 33
index 34
index 35
index 36
index 37
index 38
index 39
index 40
index 41
index 42
index 43
index 44
index 45
index 46
index 47
index 48
index 49
index 50
index 51
index 52
index 53
index 54
index 55
index 56
index 57
index 58
index 59
index 60
index 61
index 62
index 63
index 64
index 65
index 66
index 67
index 68
index 69
index 70
index 71
index 72
index 73
index 74
index 75
index 76
index 77
index 78
index 79
index 80
index 81
index 82
index 83
index 84
index 85
index 86
index 87
index 88
index 89
index 90
index 91
index 92
index 93
index 94
index 95
index 96
index 97
index 98
index 99
index 100
index 101
index 102
index 103
index 104
index 105
index 106
ind

Calculating Shapley Values:   0%|          | 0/190 [00:00<?, ?it/s]

input_ids: 8
input_mask: 8
segment_ids: 8
label_ids: 8
Iteration 1/10
_param_list 288
after backward function: passed!!


/tmp/ipython-input-3507123264.py:119: DeprecationWarning: __array_wrap__ must accept context and return_scalar arguments (positionally) in the future. (Deprecated NumPy 2.0)
  shapley_values[prev_index:index] = shapley_values[prev_index:index] + np.abs(


end of loop: passed!
shap val type:  <class 'torch.Tensor'>
Iteration 2/10


In [ ]:
model_to_save = model.module if hasattr(model, 'module') else model  # Only save the model it-self
torch.save(model_to_save, 'afr_model_dict.pt')


# for later
#https://saturncloud.io/blog/how-to-save-a-trained-model-in-pytorch/#:~:text=Save%20the%20state_dict%20only&text=using%20PyTorch's%20torch.-,save()%20function.,want%20to%20save%20the%20model.&text=The%20state_dict()%20method%20returns,parameters%20and%20their%20corresponding%20values.

In [ ]:
# rest of file
model_to_save = model.module if hasattr(model, 'module') else model  # Only save the model it-self
output_model_file1 = os.path.join("weights_shap/", "pytorch_model_lhg.bin")
torch.save(model_to_save.state_dict(), output_model_file1)

In [ ]:
# then can use the model for prompting

In [ ]:
# Prompt again to see if it is better
# https://www.kaggle.com/code/nilaychauhan/gemma-2-inference-using-pytorch

# this has a nice code for a prompt

In [ ]:
# print(len([param for param in model.parameters()]))
# print([param for param in model.parameters()])

In [ ]:
# print(len(model.state_dict()))

In [ ]:
# print(model.state_dict())

In [ ]:
# Load the template dataset from original paper (Dixon 2018)
df = pd.read_csv("african_df.csv")
shuffled_df = df.sample(frac=1, random_state=42).reset_index(drop=True) # random state in case of repetition

In [ ]:
torch.set_grad_enabled(False)

In [ ]:
tokenizer2 = AutoTokenizer.from_pretrained("google/gemma-2-2b-it")
# model2 = AutoModelForCausalLM.from_pretrained("google/gemma-2-2b-it").to(device)
model2 = torch.load("afr_model_dict.pt", weights_only = False) # will error without the "weights_only = False"

In [ ]:
prompt1 = """<start_of_turn>user You will be given a sentence and you will assign one label to each sentence.\nThe labels for the sentence can be: toxic or non-toxic.\nDo not give an explanation, just write the label.\nSentence: """
prompt3 = """\nLabel:<end_of_turn>\n<start_of_turn>model"""

In [ ]:
output_list = []

for i in range(len(shuffled_df)):
  prompt2 = shuffled_df["phrase"][i]
  if not prompt2:
    continue
  full_prompt = prompt1 + prompt2 + prompt3
  # print(full_prompt)
  # print(shuffled_df["toxicity"][i])
  inputs = tokenizer2.encode(full_prompt, return_tensors="pt", add_special_tokens=True).to(device)
  # print(i)

# Pass it in to the model and generate text
  outputs = model2.generate(input_ids=inputs, max_new_tokens=70)
  text_outputs = tokenizer2.decode(outputs[0])
  # print(text_outputs)

  if re.search(r"<start_of_turn>model: non-toxic \n<end_of_turn>", text_outputs):
    output_list.append(0)
  elif re.search(r"<start_of_turn>model: toxic \n<end_of_turn>", text_outputs):
    output_list.append(1)
  else:
    output_list.append(3)
    print(text_outputs, i)


In [ ]:
output_list

In [ ]:
final_df = pd.read_csv("african_df.csv")

In [ ]:
final_df["new model preds"] = output_list

In [ ]:
final_df.to_csv("unlearned_afr_preds.csv")

In [ ]:
tox_text_to_num = [0 if x=="nontoxic" else 1 for x in final_df["toxicity"]]
tox_text_to_num

In [ ]:
final_df["tox in nums"] = tox_text_to_num
final_df

In [ ]:
percent_correct = (final_df["tox in nums"] == final_df["new model preds"]).sum() / len(final_df)
print(percent_correct)